In [1]:
# read csv data file
import csv
f = open("module2_quiz_P1.csv")
csvfile = csv.DictReader(f)

headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

In [2]:
# define set of Segments
Segments = []
for row in table:
    if row['Segment'] != 'Cost per ad':
        Segments.append(row['Segment'])
        
# define set of Outlets
Outlets = headers[:]
Outlets.remove('Segment')
Outlets.remove('Minimum Impressions')

In [3]:
# define parameters
Cost = {}
for o in Outlets:
    Cost[o] = float(table[-1][o])

Demand = {}
for row in table:
    if row['Segment'] != 'Cost per ad':
        Demand[row['Segment']] = float(row['Minimum Impressions'])
        
Impressions = {}
for row in table:
    if row['Segment'] != 'Cost per ad':
        for o in Outlets:
            Impressions[(row['Segment'],o)] = float(row[o])

**Sets:**

Let's denote by:
- $S$ the set of segments (WWC, TTL, NEN, ILC), and
- $O$ the set of advertising outlets (R, TV1, TV2, TV3, W, M).

**Parameters:**

We'll define the following parameters:
- $C_o$: cost of an ad in outlet $o$, for each $o$ in $O$.
- $D_s$: minimum impressions for segment $s$, for each $s$ in $S$.
- $I_{so}$: impressions for segment $s$ from an ad in outlet $o$, for each $s$ in $S$ and each $o$ in $O$.

**Variables:**

We define the following variables:
- $x_{so}$: number of ads for segment $s$ to be broadcasted on outlet $o$, for each $s$ in $S$ and each $o$ in $O$.

**Objective Function:**

The objective is to minimize the total cost of the advertising campaign:

Minimize $Z = \sum_{s \in S} \sum_{o \in O} x_{so} * C_o$

**Constraints:**

Impression Constraints: For each segment $s$ in $S$, the total impressions must meet the demand:

$\sum_{o \in O} x_{so} * I_{so} \geq D_s$  for each $s$ in $S$

Web and Mobile Constraint: The total spending on web and mobile advertising should not exceed 10% of the total spending:

$\sum_{s \in S} (x_{sW} * C_W + x_{sM} * C_M) \leq 0.1 * \sum_{s \in S} \sum_{o \in O} x_{so} * C_o$

The decision variables can take any non-negative real values:

$x_{so} \geq 0$ for each $s$ in $S$ and each $o$ in $O$.


In [4]:
from docplex.mp.model import Model
import csv

# create a model
mdl = Model('Advertising_Campaign')

# decision variables, how many ads to purchase from each outlet
num_ads = mdl.continuous_var_dict([(s, o) for s in Segments for o in Outlets], name='num_ads')

# objective function, minimize total cost
mdl.minimize(mdl.sum(num_ads[s, o] * Cost[o] for s in Segments for o in Outlets))

# constraints: each segment's impressions must meet the minimum demand
for s in Segments:
    mdl.add_constraint(mdl.sum(num_ads[s, o] * Impressions[s, o] for o in Outlets) >= Demand[s], ctname='min_impression_' + s)

# constraint: web and mobile advertising combined should not exceed 10% of total cost
mdl.add_constraint(mdl.sum(num_ads[s, o] * Cost[o] for s in Segments for o in ['W', 'M']) <= 0.1 * mdl.sum(num_ads[s, o] * Cost[o] for s in Segments for o in Outlets), ctname='web_mobile_limit')

# solve the model
solution = mdl.solve()

# print the solution
if solution:
    print('Optimal advertising strategy:')
    for s in Segments:
        for o in Outlets:
            print(f'Number of ads for {s} on {o}: {num_ads[s, o].solution_value}')
else:
    print('No solution found')

# print the total cost
print('Total cost: ', mdl.objective_value)

Optimal advertising strategy:
Number of ads for WWC on R: 0
Number of ads for WWC on TV1: 0
Number of ads for WWC on TV2: 0
Number of ads for WWC on TV3: 12.5
Number of ads for WWC on W: 0
Number of ads for WWC on M: 0
Number of ads for TTL on R: 0
Number of ads for TTL on TV1: 0
Number of ads for TTL on TV2: 0
Number of ads for TTL on TV3: 41.20681920143562
Number of ads for TTL on W: 0
Number of ads for TTL on M: 28.818977119784652
Number of ads for NEN on R: 16.666666666666668
Number of ads for NEN on TV1: 0
Number of ads for NEN on TV2: 0
Number of ads for NEN on TV3: 0
Number of ads for NEN on W: 0
Number of ads for NEN on M: 0
Number of ads for ILC on R: 0
Number of ads for ILC on TV1: 0
Number of ads for ILC on TV2: 0
Number of ads for ILC on TV3: 34.666666666666664
Number of ads for ILC on W: 0
Number of ads for ILC on M: 0
Total cost:  25937.07940780619


## Linear Programming Model Formulation

This problem is a form of **Mixing and Blending Problem**. We are given different advertising outlets and we need to find the optimal combination of these outlets to meet the minimum impressions for each market segment while minimizing the total cost of the campaign. This is similar to creating a mutual fund where the base products are different investments, or designing a chemical mix where the base products are different chemicals.

**Let us denote:**

- $S$ as the set of market segments.
- $O$ as the set of advertising outlets.
- $D_s$ as the minimum required impressions for market segment $s \in S$.
- $C_o$ as the cost of purchasing an ad in outlet $o \in O$.
- $I_{so}$ as the number of impressions that an ad in outlet $o \in O$ generates for market segment $s \in S$.
- $x_{so}$ as the decision variable representing the number of ads for segment $s \in S$ to be broadcasted on outlet $o \in O$.

**Objective Function:**

Minimize the total cost of the advertising campaign:

$$\min \sum_{s \in S} \sum_{o \in O} C_o \cdot x_{so}$$

**Constraints:**

1. **Demand Constraint:** For each market segment $s \in S$, the total impressions must meet the demand:

$$\sum_{o \in O} I_{so} \cdot x_{so} \geq D_s, \quad \forall s \in S$$

2. **Budget Constraint:** The total spending on web and mobile advertising should not exceed 10% of the total spending:

$$\sum_{s \in S} (C_W \cdot x_{sW} + C_M \cdot x_{sM}) \leq 0.1 \cdot \sum_{s \in S} \sum_{o \in O} C_o \cdot x_{so}$$

3. **Non-Negativity Constraint:** For each market segment $s \in S$ and advertising outlet $o \in O$, the number of ads should be non-negative:

$$x_{so} \geq 0, \quad \forall s \in S, \forall o \in O$$

This linear programming model allows us to optimize the allocation of resources (advertising outlets) while considering constraints (minimum impression requirements and spending limits). 


In [5]:
# create model
mdl = Model()

# define decision variables
x = mdl.continuous_var_matrix(keys1=Segments, keys2=Outlets, name=lambda ns: "x_%s_%s" % ns)

# add constraints
for s in Segments:
    mdl.add_constraint(mdl.sum(Impressions[(s,o)]*x[s,o] for o in Outlets) >= Demand[s])

# add budget constraint for web and mobile advertising
mdl.add_constraint(mdl.sum(Cost[o]*x[s,o] for s in Segments for o in ['W','M']) <= 0.1*mdl.sum(Cost[o]*x[s,o] for s in Segments for o in Outlets))

# define objective
mdl.minimize(mdl.sum(Cost[o]*x[s,o] for s in Segments for o in Outlets))

# define KPIs
total_cost = mdl.sum(Cost[o]*x[s,o] for s in Segments for o in Outlets)
mdl.add_kpi(total_cost, "Total Cost")

for s in Segments:
    total_impressions = mdl.sum(Impressions[(s,o)]*x[s,o] for o in Outlets)
    mdl.add_kpi(total_impressions, "Total Impressions " + s)

total_web_mobile_spending = mdl.sum(Cost[o]*x[s,o] for s in Segments for o in ['W','M'])
mdl.add_kpi(total_web_mobile_spending, "Total Spending on Web and Mobile Advertising")

# solve model
mdl.solve()

# print KPIs and decision variable values
mdl.report()

print(" ")

for s in Segments:
    for o in Outlets:
        print("x[%s,%s] = %f" % (s, o, x[s,o].solution_value))

* model docplex_model1 solved with objective = 25937.079
*  KPI: Total Cost                                   = 25937.079
*  KPI: Total Impressions WWC                        = 50000.000
*  KPI: Total Impressions TTL                        = 70000.000
*  KPI: Total Impressions NEN                        = 25000.000
*  KPI: Total Impressions ILC                        = 52000.000
*  KPI: Total Spending on Web and Mobile Advertising = 2593.708
 
x[WWC,R] = 0.000000
x[WWC,TV1] = 0.000000
x[WWC,TV2] = 0.000000
x[WWC,TV3] = 12.500000
x[WWC,W] = 0.000000
x[WWC,M] = 0.000000
x[TTL,R] = 0.000000
x[TTL,TV1] = 0.000000
x[TTL,TV2] = 0.000000
x[TTL,TV3] = 41.206819
x[TTL,W] = 0.000000
x[TTL,M] = 28.818977
x[NEN,R] = 16.666667
x[NEN,TV1] = 0.000000
x[NEN,TV2] = 0.000000
x[NEN,TV3] = 0.000000
x[NEN,W] = 0.000000
x[NEN,M] = 0.000000
x[ILC,R] = 0.000000
x[ILC,TV1] = 0.000000
x[ILC,TV2] = 0.000000
x[ILC,TV3] = 34.666667
x[ILC,W] = 0.000000
x[ILC,M] = 0.000000


The result of our linear programming optimization for the advertising campaign shows that the optimal allocation of ads among the outlets would minimize the total cost to $25937.08. This plan meets all the minimum impression requirements for each segment and follows the constraint that no more than 10% of the overall spending can be spent on web and mobile advertising.

Here's a breakdown of the optimal solution:

- The "Women With Children (WWC)" segment should be targeted with 12.5 ads on TV3. 

- The "Twenty-something trend leaders (TTL)" segment should be targeted with approximately 41.2 ads on TV3 and 28.8 ads on Mobile.

- The "Nurturing Empty Nesters (NEN)" segment should be targeted with about 16.67 ads on Radio.

- The "Insistent Little Children (ILC)" segment should be targeted with approximately 34.67 ads on TV3.

No ads are purchased on the website (W), as per the optimal solution.

This allocation meets the minimum impressions required for each segment while minimizing the total cost. Moreover, the total spending on Web and Mobile Advertising was kept within the prescribed limit, amounting to $2593.71, which is approximately 10% of the total cost.

It's important to note that these results are based on the specific costs, impressions per ad, and minimum impressions provided in the problem. If these parameters change, the optimal solution may vary. Also, the fractional values for the number of ads indicate that this is a mathematical optimization, and in a real-world scenario, the company would need to round these numbers, probably upwards, to whole numbers.
